In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestRegressor
import xgboost as xg
from xgboost import XGBRegressor
import os
import torch
from torch import optim
from torch import nn
from sklearn.metrics import r2_score
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Data Set/imputed_train (1).csv')
y_train = train.pop('Rainfall').to_numpy()
X_train = train.to_numpy()
test = pd.read_csv('/content/drive/MyDrive/Data Set/imputed_test.csv')
y_test = test.pop('Rainfall').to_numpy()
X_test = test.to_numpy()

In [ ]:
def KNNr(X_test, X_train, y_train):
  # KNNRegressor
  KNN = KNeighborsRegressor(n_neighbors = 55)
  KNN.fit(X_train, y_train)
  KNeighborsRegressor(n_neighbors = 55)
  y_hat_k = KNN.predict(X_test)
  return y_hat_k

In [ ]:
def RFr(X_test, X_train, y_train):
  # RandomForestRegressor
  regr = RandomForestRegressor(n_estimators = 300, max_depth = 64, random_state = 42)
  regr.fit(X_train, y_train)
  y_hat_r = regr.predict(X_test)
  return y_hat_r

In [ ]:
def XGBr(X_test, X_train, y_train):
  # XGBoost Regressor
  my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05,random_state=42)
  my_model.fit(X_train, y_train)
  y_hat_xg= my_model.predict(X_test)
  return y_hat_xg

In [ ]:
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyNeuralNetwork, self).__init__()
        #Define the NN Architecture
        self.layer1 = nn.Sequential(
            nn.Linear(18, 1024),
            nn.ReLU()
        )
        self.layer2= nn.Sequential(
            nn.Linear(1024,256),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU()
        )
        self.layer4= nn.Sequential(
            nn.Linear(64, 16),
            nn.ReLU()
        )
        self.layer5= nn.Sequential(
            nn.Linear(16,4),
            nn.ReLU()
        )
        self.layer6 = nn.Linear(4,1)

    def forward(self, x):
        # Define the forward pass
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return x

In [ ]:
def NN(X_test, X_train, y_train):
  model=MyNeuralNetwork()

  # Compute the RMS loss
  criterion = nn.MSELoss()  # Mean Squared Error loss
  optimizer = optim.Adam(model.parameters(), lr=0.03)

  epochs = 100
  num_samples = X_train.shape[0]

  # Training loop
  for epoch in range(epochs):
      total_loss = 0.0
      y_train = y_train.view(-1,1)
      for i in range(0, num_samples, 5000):
          inputs = X_train[i:i+5000]
          targets = y_train[i:i+5000]


          optimizer.zero_grad()  # Zero the gradients
          outputs = model(inputs)
          outputs = outputs.view(-1,1)
          # Compute the loss
          loss = criterion(outputs, targets)

          # Backpropagation
          loss.backward()

          # Update weights

          optimizer.step()

          # Add the batch loss to the total loss for this epoch
          total_loss += loss.item()

      # Print the average loss for this epoch
      average_loss = total_loss / num_samples

      model.train()  # Set the model back to training mode
  with torch.no_grad():
    y_hat_nn=model(X_test)
    return y_hat_nn

In [ ]:
ep=1e-3
batch_size=500
for i in (0, X_test.shape[0], batch_size):
  input=X_test[i:i+batch_size]
  Y1=KNNr(input, X_train, y_train)
  Y2=RFr(input, X_train, y_train)
  Y3=XGBr(input, X_train, y_train)
  Y4=NN(input, X_train, y_train)
  Y_hat=W1*Y1+W2*Y2+W3*Y3+W4*Y4
  Y_hat/=(W1+W2+W3+W4)
  # Update Weights according to metric
  r2=[r2_score(Y1, y_test[i:i+batch_size]), r2_score(Y2, y_test[i:i+batch_size]), r2_score(Y3, y_test[i:i+batch_size]), r2_score(Y4, y_test[i:i+batch_size])]
  r2=r2.to_numpy()
  r2=(r2-r2.mean())/r2.std()
  W1=W1*torch.exp(r2[0]*ep)
  W2=W2*torch.exp(r2[1]*ep)
  W3=W3*torch.exp(r2[2]*ep)
  W4=W4*torch.exp(r2[3]*ep)
  # Update Training Data
  X_train.append(input)
  y_train.append(y_test[i:i+batch_size])